In [1]:
#!pip install modestga --upgrade --quiet
#!pip install deap --upgrade --quiet
import warnings
warnings.filterwarnings("ignore")

In [2]:


import numpy as np
import pandas as pd
import random
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.optimize import curve_fit
import time


# Start timer
start_time = time.perf_counter()



In [3]:
#===========================================================
#                   Read Excel File (Node_Property)
#===========================================================

file = pd.ExcelFile('Nodes-Property.xlsx')
df1=pd.read_excel(file,'Node_Property')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
Node_Name= data[:,0]
Memory= data[:,1]                                # Accessible memory size of CNk for running tasks
tsk= data[:,2]                                   # Switch time between two jobs for CNk 
#print(tsk)
#===========================================================
#                   Read Excel File 
#===========================================================

file = pd.ExcelFile('results_bacteria30g_4nodes.xlsx')
df1=pd.read_excel(file,'General-Results')
data=df1[df1.columns[:]].to_numpy()
data = np.unique(data,axis=0)
#print ('data: \n',data)
nodename= data[:,0]
x= data[:,1]
y= data[:,2]


nodename_item =list(dict.fromkeys(nodename))
V = len(nodename_item)  # The number of nodes of the cluster

#===========================================================
#                   Read Excel File 
#===========================================================
#genomes_df= pd.read_csv('overview-test.xlsx')
genomes_df= pd.read_excel('overview_results_bacteria30g.xlsx')

#===========================================================
#                   Initializing Variables
#===========================================================

#CNk=0                             # kth node of the cluster 
#Memk=0                            # Accessible memory size of
                                   # Switch time between two jobs for CNk
n=len(genomes_df.GenomeName)       # The number of genomes of the reference genome
Gen= genomes_df.GenomeName         # 
Gen= Gen[0:n]                      # The set of genomes of the reference genome
Si=genomes_df.Size                 # 
Si= Si[0:n]                        # Size of gi
#J = n                           # The maximum number of Jobs
J = V

poly_log1=[]
poly_log2=[]
poly_log3=[]
#poly_log4=[]
Dictionary_log={}
FinalOutput_log={}
#===========================================================
#                   Initializing Genetic Algorithm parameters 
#===========================================================

solution =[]

population_number=20
MutationRate= 0.4
CrossoverRate= 1 - MutationRate
maxIter = 10
MutationNumber = round (population_number* MutationRate);
CrossoverNumber = round (population_number * CrossoverRate);


In [4]:
#================================================================================================
#                  f(X)=Ln aX+b    ****** Ln(x)=np.log(x)
#================================================================================================
           
def f_log1 (x, a,b):
    p=(x*a+b)
   # print(len(p))
    t=np.log(p)
    return t


#================================================================================================
#                  f(X)=Ln aX^2 + bX + c
#================================================================================================

def f_log2 (x, a,b,c):
    t=np.log((x**2)*a +(x*b)+c)
    return t



#================================================================================================
#                  f(X)=Ln aX^3 + bX^2 + cX + d
#================================================================================================
def f_log3 (x, a, b,c, d):
    t=np.log((x**3)*a +(x**2)*b +(x*c)+d)
    return t


#================================================================================================
#                  f(X)=Ln aX^4 + bX^3+ cX^2 + dX + e
#================================================================================================
''' def f_log4 (x, coefficient):
    t=np.log(x**4*coefficient[0] +x**3*coefficient[1] +x**2*coefficient[2] +x*coefficient[3]+coefficient[4])
    return t   '''
#================================================================================================
#                  f_logk(X)
#================================================================================================
def fk (k,x, coefficients):
    if k==1:
        return f_log1(x, *coefficients)
    if k==2:
         return f_log2(x, *coefficients)
    if k==3:
         return f_log3(x, *coefficients)
   # if k==4:
      #   return f_log4(x, coefficients)

 #================================================================================================
#  Find The Best or Simplest Logarithmic Function Based on Mean Square Error 
#================================================================================================

Estimated_value_by_degree_log1 =[]
Estimated_value_by_degree_log2 =[]
Estimated_value_by_degree_log3 =[]
#Estimated_value_by_degree_log4 =[]
MSE_log=[]
BestLogarithmicDegree=[[]]
for i in range (V):
  MSE_log=[]
  #print('i : ', i)
  a_log= data ==nodename_item[i]
  nodenamep_log= nodename[a_log[:,0]]
  xp_log= x[a_log[:,0]]
  yp_log= y[a_log[:,0]]
  #print('Original X for node name of {} is: {}'.format(int(nodenamep[0]),xp))
  #print('Original Y for node name of {} is: {}'.format(int(nodenamep[0]),yp))
  #print('\n')

  #======================================================
  # Logarithmic Equation Degree of 1  & MSE 
  #======================================================
  coefficient_log1, covariance1= curve_fit(f_log1, xp_log, yp_log, bounds=(0, [99999., 999999.]))
  poly_log1.append(coefficient_log1)
  #print('coefficient_log1= ',coefficient_log1)
  Estimated_y_by_degree_log1 = f_log1(xp_log,*coefficient_log1)
  #print(np.isnan(Estimated_y_by_degree_log1))
    # logarithim of a negative value is nun, in this case we replac the result with a large number like 9999
  np.nan_to_num(Estimated_y_by_degree_log1, copy = False, nan = 9999)
  #print(np.isnan(Estimated_y_by_degree_log1))
  #print('Estimated Y by Degree of 1= ',  Estimated_y_by_degree_log1)
  MSE_log.append(mean_squared_error(yp_log,Estimated_y_by_degree_log1))


  #======================================================
  # Logarithmic Equation Degree of 2 & MSE
  #======================================================
  if (len(xp_log)>1):
      coefficient_log2, covariance2= curve_fit(f_log2, xp_log, yp_log, bounds=(0, [99999., 999999.,999999. ]))
      #coefficient_log2= np.polyfit(xp_log,np.exp(yp_log),2);
      poly_log2.append(coefficient_log2)
      #print('coefficient2= ',coefficient2)
      Estimated_y_by_degree_log2=f_log2(xp_log,*coefficient_log2);
      #print('Estimated Y by Degree of 2= ', Estimated_y_by_degree_2)
      np.nan_to_num(Estimated_y_by_degree_log2, copy = False, nan = 9999)
      MSE_log.append( mean_squared_error(yp_log,Estimated_y_by_degree_log2))


  #======================================================
  # Logarithmic Equation Degree of 3 & MSE
  #======================================================
  if (len(xp_log)>2):
      coefficient_log3, covariance3= curve_fit(f_log3, xp_log, yp_log, bounds=(0, [99999., 999999.,999999. ,999999. ]))
      #coefficient_log3= np.polyfit(xp_log,np.exp(yp_log),3)
      poly_log3.append(coefficient_log3)
      #print('coefficient3= ',coefficient3);
      Estimated_y_by_degree_log3=f_log3(xp_log,*coefficient_log3)
      #print('Estimated Y by Degree of 3= ',Estimated_y_by_degree_3 )
      np.nan_to_num(Estimated_y_by_degree_log3, copy = False, nan = 9999)
      MSE_log.append( mean_squared_error(yp_log,Estimated_y_by_degree_log3))


  #======================================================
  # Logarithmic Equation Degree of 4 & MSE
  #======================================================
  ''' if (len(xp_log)>3):
      coefficient_log4= np.polyfit(xp_log,np.exp(yp_log),4)
      poly_log4.append(coefficient_log4);
      #print('coefficient4= ',coefficient4)
      Estimated_y_by_degree_log4= f_log4(xp_log,coefficient_log4)
      #print('Estimated Y by Degree of 4= ',Estimated_y_by_degree_4)
      np.nan_to_num(Estimated_y_by_degree_log4, copy = False, nan = 9999)
      MSE_log.append( mean_squared_error(yp_log,Estimated_y_by_degree_log4))  '''


  #======================================================
  # Find Minimum of MSE and its index
  #======================================================
  #print('\n')
  #print ("MSE = ", MSE)
  MinMSE_log =min(MSE_log)
  #print("Min(MSE) = ", MinMSE)
  Dictionary_log={}
  Dictionary_log["NodeName"] = int( nodenamep_log[0])
  Dictionary_log["BestLogarithmicDegree"]=MSE_log.index(MinMSE_log)+1
  if (MSE_log.index(MinMSE_log)==0):
      Dictionary_log["Coefficients"]=coefficient_log1
  elif (MSE_log.index(MinMSE_log)==1):
      Dictionary_log["Coefficients"]=coefficient_log2
  elif (MSE_log.index(MinMSE_log)==2):
      Dictionary_log["Coefficients"]=coefficient_log3
  #elif (MSE_log.index(MinMSE_log)==3):
     # Dictionary_log["Coefficients"]=coefficient_log4
        
  FinalOutput_log[i]= Dictionary_log
  BestLogarithmicDegree.append([nodenamep_log[0],MSE_log.index(MinMSE_log)+1])

  #print('The Best and simplest Ploynomial Degree for node name {} is: {}'.format(int(nodenamep[0]),MSE.index(MinMSE)+1))
  #print('\n')
  #print ('*****************************************************************************************')
  #print('\n')
#======================================================
# Print result for Best Polynomaial Equation Degree Based MSE
#======================================================
#print('\n')
#print('==========================================================================================')
#print('           Best Ploynomial Degree (Based on MSE): \n', BestLogarithmicDegree)
#print('==========================================================================================')
#print('\n')
#print('\n')
#print('\n')

FinalOutput_log
       




{0: {'NodeName': 213,
  'BestLogarithmicDegree': 3,
  'Coefficients': array([ 99999., 999999., 999999., 999999.])},
 1: {'NodeName': 219,
  'BestLogarithmicDegree': 3,
  'Coefficients': array([ 99999., 999999., 999999., 999999.])},
 2: {'NodeName': 223,
  'BestLogarithmicDegree': 3,
  'Coefficients': array([ 99999., 999999., 999999., 999999.])},
 3: {'NodeName': 228,
  'BestLogarithmicDegree': 3,
  'Coefficients': array([ 99999., 999999., 999999., 999999.])}}

In [5]:
def jobs2cluster(S):
    AssignedJobs2Cluster={}
    for i in range (V):
        AssignedJobs2Cluster[i]=[]
    for j in range (n):
        Jobj=int(S[0][j])
        CNk=int(S[1][j])
        AssignedJobs2Cluster[CNk].append(Jobj)
    return (AssignedJobs2Cluster)

In [6]:
def genome2cluster(SS):
    g2c={}
    for i in range (V):
        g2c[i]=[]
    for j in range (n):
        Genomej=j
        CNk=int(SS[1][j])
        g2c[CNk].append(Genomej)
    return(g2c.copy())

In [7]:
"""Feasibility function for the Memk. Returns True if feasible False
    otherwise."""
def feasibleMemk(sol):
 
    
    M=np.zeros((V,J))

        
   
    for i in range (n):
        jobj= int(sol[0][i])
        CNk = int(sol[1][i])
        
        item= Si[i]
        q= len(item)
        if item[q-1]== 'K' or item[q-1]== 'k' :
            DigitSize=item[:-1]
            
            M[CNk][jobj]+= (float(DigitSize)/1000)
        elif  item[q-1]== 'M' or item[q-1]== 'm' :
            DigitSize=item[:-1]
            M[CNk][jobj]+= (float(DigitSize))
        
    for k in range (V):
        for j in range (J):
            if  Memory[k]< 3*M[k][j]:
                return False
    return True
    
  

In [8]:
def ComputeJobSize(g2j):
    sm=[]
    for j in range (J):
            sm.append(0.0)
    for j in range (J):

            GenomesNumber= g2j[j]
            m= len(GenomesNumber)
            #print(" GenomesNumber:",  GenomesNumber)

            #extract memory size of each genome belonged to Job j
            MemoryItems = (Si[GenomesNumber])
            #print (' MemoryItems :',  MemoryItems )
            for i in GenomesNumber:
                item= Si[i]
                #print ('Genome Size {} is: {}'.format(i,Si[i]))
                q= len(item)
                if item[q-2]== 'K' or item[q-2]== 'k' :
                    DigitSize=item[:-1]
                    sm[j]+= float(DigitSize)/1000
                elif  item[q-2]== 'M' or item[q-2]== 'm' :
                    DigitSize=item[:-2]
                    sm[j]+= float(DigitSize)
    print('Compute job size=',sm.copy() )
    return sm.copy()

In [9]:
def RunningTime (Sl,g2cn):
    tk= np.zeros((V,1))
    J2C=jobs2cluster(Sl)
    G2J=Genomes2Jobs(Sl)
    JS= ComputeJobSize(G2J) # Job Size
    for k in range (V):
        NeedMemory=0
        for j in range (len(np.unique(J2C[k]))):
            #print ('Cluster Number is: {},  Job Number is {}, and Job size is {}. \n '.format (k,J2C[k][j],JS[J2C[k][j]]))
            NeedMemory=JS[J2C[k][j]]
            if(NeedMemory == 0) :
                continue
            else:
                tk[k]+=fk(FinalOutput_log[k]['BestLogarithmicDegree'],NeedMemory,FinalOutput_log[k]['Coefficients'])
    for i in range (V):
        GenomesNumber= g2cn[i]
        m= len(np.unique(J2C[i]))
        tk[i]+= ((m-1)*tsk[i])
    print('tk=',max(tk))    
    return max(tk)
            
 
          
        
    
       

In [10]:
def Genomes2Jobs(SL):
    Ge2Jo={}
    for i in range (n):
        Ge2Jo[i]=[]
    for j in range (n):
        Genomej=j
        Jobj=int(SL[0][j])
        Ge2Jo[Jobj].append(Genomej)
    return(Ge2Jo.copy())

In [11]:
def clusters2job(solu):
    C2Job={}
    for i in range (n):
        C2Job[i]=[]
    for j in range (n):
        Jobj=int(solu[0][j])
        CNk=int(solu[1][j])
        C2Job[Jobj].append(CNk)
      
    return(C2Job.copy())


In [12]:
import random
def AddressJobs2ClustersProblem(SOL):
    Temp= clusters2job(SOL.copy())
    t= len(Temp)
    for j in range (len(Temp)):
        if len(Temp[j])>1:
            newCNkIndex= random.randint(0,len(Temp[j])-1)
            newCNk= Temp[j][newCNkIndex]
            for i in range (n):
                if SOL[0][i]== j:
                    SOL[1][i]=newCNk
    return SOL.copy()
                
           
        
    
    
    


In [13]:
def equalSols(S1,S2):
    for j in range(len(S1)):
        if (S1[1][j]!= S2[1][j] or S1[0][j]!= S2[0][j] ):
                return False
    return True

In [14]:
def isRepeated(POp,Sol):
    index= False
    newPop=[]
    for i in range(len(POp)):
        newPop.append((POp[i].copy()))
    newPop=np.array(newPop.copy())
    N= len(POp)
    for i in range(N):
        if (equalSols(Sol.copy(),newPop[i,:,:].copy())):
                index =True
    return index

In [15]:
def arrangeSolutions(POPU):
    newFit = []
    newPop=np.empty((len(POPU), 2,n))
    for i in range(len(POPU)):
         if (feasibleMemk(POPU[i].copy())==True):
           
            newPop[i]=((POPU[i].copy()))
            SolvedSol= AddressJobs2ClustersProblem(POPU[i].copy())   
            G2CNK= genome2cluster(SolvedSol.copy())
            Fit= RunningTime(SolvedSol.copy(),G2CNK)
            newFit.append(Fit)
    
    return newPop.copy(),newFit.copy()
        
    

In [16]:
def AddNewSol(POPulation,Sol,FITS,Fit):
    index= isRepeated(POPulation.copy(),Sol.copy())
    L= len( POPulation.copy())
    if(index == False ):
        L=L+1
        POPulation= np.append(POPulation.copy(),Sol.copy())
        POPulation= POPulation.reshape(L,2,n)
        FITS.append(Fit)
        
    return POPulation.copy(),FITS.copy()

In [17]:
def SortPopulation(poP, FS,Npop):
    
    newFit = []    
    newPop=[]
    
    for i in range(len(poP)):
        newPop.append((poP[i].copy()))
        newFit.append(FS[i][0].copy())
    s = np.array(newFit.copy())
   
    sort_index = np.argsort(newFit.copy())
    for i in range(len(poP)):
        newPop[i]=list( poP[sort_index[i]].copy())
    FS.sort()
    
    return newPop.copy(),FS.copy(),len(newPop.copy())

In [18]:

def CreatePopulation (n):
    #population[i][j][k] ==> i th population 
    #[row0] ==> Job of solution [.......]
    #[row1] ==> CNk of solution [.......]
    Npop=1
    NS= np.zeros((2,n))
    for i in range (n):
            NS[0,i]=int(random.randint(0,J-1))# Random Job  number
            NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
    NS= AddressJobs2ClustersProblem(NS.copy())   
    Sol= genome2cluster(NS.copy())
        
    #JobSize = totalMemk(Sol)
    if (feasibleMemk(NS.copy())==True):
            POPULATION= np.empty((1, 2,n))
            for i in range(Npop):
                 POPULATION[0]= NS.copy()
            Fit= RunningTime(NS.copy(),Sol)
    else:
            while(feasibleMemk(NS.copy())==False):
                NS= np.zeros((2,n))
                for i in range (n):
                    NS[0,i]=int(random.randint(0,J-1))# Random Job  number
                    NS[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
                NS= AddressJobs2ClustersProblem(NS.copy())   

                Sol= genome2cluster(NS.copy())

                #JobSize = totalMemk(Sol)
                
                
                if (feasibleMemk(NS.copy())==True):
                    
                    POPULATION= np.empty((1, 2,n))
                    for j in range(n):
                        POPULATION[0]= NS.copy()
                    Fit= RunningTime(NS.copy(),Sol)
            
    return POPULATION.copy(),Fit.copy()
        
    

In [19]:
def Mutation(PP,FITS):   
    P=PP.copy()
    FF=FITS.copy()
    for iMut in range(MutationNumber):
            Npop= len(PP)
            SelectedSolutionIndex= random.randint(0,len(P)-1)
            SelectedSolution= (P[SelectedSolutionIndex]).copy()
            #print("SelectedSolution:\n", SelectedSolution)

            SelectedGenomeIndex1 = random.randint(0,n-1)
            SelectedGenomeIndex2 = random.randint(0,n-1)

            tempJ=SelectedSolution[0][SelectedGenomeIndex1]
            tempCNk=SelectedSolution[1][SelectedGenomeIndex1]
            SelectedSolution[0][SelectedGenomeIndex1]= SelectedSolution[0][SelectedGenomeIndex2].copy()
            SelectedSolution[1][SelectedGenomeIndex1]= SelectedSolution[1][SelectedGenomeIndex2].copy()
            SelectedSolution[0][SelectedGenomeIndex2]= tempJ
            SelectedSolution[1][SelectedGenomeIndex2]= tempCNk
            MutedSolution = AddressJobs2ClustersProblem(SelectedSolution.copy())
            Sol= genome2cluster(MutedSolution.copy())
            
           


            if (feasibleMemk(MutedSolution.copy())==True and  isRepeated(P.copy(),MutedSolution.copy()) ==False):
                #print('Mut len:',len(MutedSolution))
                F=( RunningTime(MutedSolution.copy(),Sol.copy()))
                P, FF= AddNewSol(P.copy(), MutedSolution.copy(),FITS.copy(),F) 
                if len(P)!= len(FF):
                    P,F= arrangeSolutions(P)
                    #print("Pop:\n", PP)
                    #print("!!!Mutelen(P):",len(P))
                    #print ("len Fit",len(F))
    return P, FF



In [20]:
def Crossover (POP,FITN):
    PoP=POP.copy()
    FiT=FITN.copy()
    for iCross in range(CrossoverNumber):
            Npop= len(PoP)
            SelectedSolutionIndex1= random.randint(0,len(PoP)-1)
            SelectedSolutionIndex2= random.randint(0,len(PoP)-1)

            SS1= PoP[SelectedSolutionIndex1].copy()
            SS2= PoP[SelectedSolutionIndex2].copy()

            SelectedGenomeIndex = random.randint(0,n-1)
            temp=  SS2[SelectedGenomeIndex:n-1].copy()
            SS2[SelectedGenomeIndex:n-1]= SS1[SelectedGenomeIndex:n-1].copy()
            SS1[SelectedGenomeIndex:n-1]= temp.copy()


            SS1=AddressJobs2ClustersProblem(SS1.copy())
            Sol1= genome2cluster(SS1.copy())



            #JobSize = totalMemk(Sol1)
            if (feasibleMemk(SS1)==True and isRepeated(PoP.copy(),SS1.copy()) ==False):
                #print('Cross1 len:',len(SS1))
                F=(RunningTime(SS1.copy(),Sol1.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS1.copy(),FiT.copy(),F)  
                


            SS2=AddressJobs2ClustersProblem(SS2)
            Sol2= genome2cluster(SS2)
            #JobSize = totalMemk(Sol2)
            if (feasibleMemk(SS2.copy())==True and isRepeated(PoP.copy(),SS2.copy()) ==False):
                #print('Croos2 len:',len(SS2))
                F=(RunningTime(SS2.copy(),Sol2.copy()))
                PoP,FiT= AddNewSol(PoP.copy(), SS2.copy(),FiT.copy(),F) 
    if (len(PoP)!= len(FiT)):
        PoP,FiT= arrangeSolutions(PoP.copy())
        #print("Pop:\n", PP)
        #print("!!!Crosslen(P):",len(PoP))
        #print ("len Fit",len(FiT))
    return PoP.copy(),FiT.copy()

In [21]:
#==================================================================
#                 Genetic Algorithm 
#==================================================================
    
def GA_Scheduling():
    #==================================================================
    #                 Create Initial Population
    #===================================================================
    Population=[]
    MaxPopN= 1000
    Npop=0  # number of added acceptable solutions to population
    fits=[]

    Population,Fit= CreatePopulation (n)
    fits.append(Fit)
    print('initial Population',Population)
    while (Npop <population_number):
        for j in range (population_number):
            NSOL= np.zeros((2,n))
            for i in range (n):
                NSOL[0,i]=int(random.randint(0,J-1))# Random Job  number
                NSOL[1,i]=int(random.randint(0,V-1)) # Random Cluster Node number
            NSOL= AddressJobs2ClustersProblem(NSOL)   

            Sol= genome2cluster(NSOL)
            #JobSize = totalMemk(Sol)
            if (feasibleMemk(NSOL)==True and isRepeated(Population.copy(),NSOL.copy()) ==False):
                Fit= (RunningTime(NSOL,Sol))
                Population, fits= AddNewSol(Population.copy(),NSOL.copy(),fits.copy(), Fit)
                Npop+=1
    #print('initpop:',Population)
   
    #==================================================================
    #                 Genetic Algorithm 
    #===================================================================
    for iteration in range(maxIter):
        changed=False

        changed=False
        nMut=0   # number of acceptable Muted solutions 
        nCross=0 # number of acceptable Crossover solutions 
        Npop= len(Population)

        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                 Mutation
        #===================================================================
                
        #print ('Mut1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits= Mutation(Population.copy(),fits.copy())
        #print ('Mut2:')
        #print(len(Population))
        #print(len(fits))
        #for i in range (len (Population)):
            #print(Population[i])
        #==================================================================
        #                Crossover
        #===================================================================
        #print ('Cros1:')
        #print(len(Population))
        #print(len(fits))
        Population,fits=Crossover (Population.copy(),fits.copy())
        #print ('Cros2:')
        #print(len(Population))
        #print(len(fits))

        #==================================================================
        #                Add Muted Solutions to Population
        #===================================================================

        ''' print ('Pop1:')
        for i in range (len (Population)):
            print(Population[i])'''
        Population, fits,Npop=SortPopulation(Population.copy(), fits.copy(), Npop-1)    
        Population,fits= arrangeSolutions(Population.copy())
        
        '''print ('Pop2:')
        for i in range (len (Population)):
            print(Population[i])'''
        ''' print("\n***************************************************************************\n")
        print('Best Solution is of iteration of {} is:\n{} '.format(iteration,Population[0].copy() ))
        print("-----------------------------------------------------------------------------\n") '''
    
    return Population.copy(), fits.copy()

P, F= GA_Scheduling()

Compute job size= [7794.899999999994, 7890.999999999998, 7594.399999999994, 7412.200000000015]
tk= [88.63946156]
initial Population [[[2. 2. 3. ... 0. 0. 2.]
  [1. 1. 1. ... 2. 2. 1.]]]
Compute job size= [7991.8000000000175, 7804.8, 7231.999999999991, 7663.899999999997]
tk= [92.69409686]
Compute job size= [8073.799999999985, 7531.1, 7427.099999999993, 7660.500000000018]
tk= [91.00658406]
Compute job size= [7842.700000000009, 7501.999999999993, 7813.499999999997, 7534.300000000001]
tk= [86.68804205]
Compute job size= [7621.200000000003, 7678.500000000005, 7580.900000000003, 7811.8999999999905]
tk= [92.628791]
Compute job size= [7565.700000000001, 7691.999999999995, 7732.999999999986, 7701.800000000013]
tk= [90.67021823]
Compute job size= [7879.300000000006, 7712.300000000001, 7611.50000000001, 7489.399999999986]
tk= [92.86033404]
Compute job size= [7923.499999999998, 7332.999999999997, 7784.1, 7651.9000000000015]
tk= [92.42939679]
Compute job size= [7637.3, 7680.400000000001, 7931.19999

In [22]:
BestSolution = P[0]
ClusterName= np.unique(nodename)
#print('The best solution:\n')
for i in range(n):
    j= int(BestSolution[0][i])
    k= int(BestSolution[1][i])
    #print( "Genome {} is allocated to Job Number {} on Cluster Node Number {}.".format(i,j,ClusterName[k]))

print('\n and its fitness function is:\n',(F[0]))


 and its fitness function is:
 [38.38630923]


In [23]:
J2C= jobs2cluster(BestSolution)
EmptyClusters=[]
for k in range(V):
    if (J2C[k]!= []):
        print('Cluster {} includes a subset of Job numbers:\n{}'.format(ClusterName[k],np.unique( J2C[k])))
    else:
        EmptyClusters.append(ClusterName[k])
print("================================================================================")
print ("Empty Clusters are:", (EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

#print('Size Job number{} = {}'.format(np.unique( J2C[k]), )

Cluster 213.0 includes a subset of Job numbers:
[2]
Cluster 219.0 includes a subset of Job numbers:
[0]
Cluster 223.0 includes a subset of Job numbers:
[1]
Cluster 228.0 includes a subset of Job numbers:
[3]
Empty Clusters are: []
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [24]:

G2J= Genomes2Jobs(BestSolution)
JobSize=ComputeJobSize(G2J)
EmptyJobs=[]
for i in range(J):
    if (G2J[i]!= []):
        print('Job {} includes a subset of genomes numbers:{}'.format(i,G2J[i]))
    else:
        EmptyJobs.append(i)
print("================================================================================")
print ("Empty Jobs are:", EmptyJobs)
print ("The number of Empty Jobs is:", len(EmptyJobs))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
for k in range(J):
    print('Size Job number{} = {}'.format(k,JobSize[k]))
print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
print('running time of the best solution=' )

Compute job size= [7666.999999999992, 7764.899999999994, 7633.000000000006, 7627.60000000001]
Job 0 includes a subset of genomes numbers:[0, 10, 22, 24, 27, 30, 32, 34, 39, 44, 45, 46, 51, 54, 55, 56, 72, 73, 77, 82, 86, 88, 96, 106, 115, 116, 117, 119, 121, 128, 131, 132, 133, 146, 151, 154, 155, 161, 165, 169, 175, 177, 182, 188, 198, 212, 213, 216, 220, 222, 227, 228, 234, 235, 236, 241, 242, 250, 255, 259, 260, 261, 266, 269, 274, 276, 282, 288, 298, 312, 318, 319, 320, 323, 326, 329, 332, 335, 341, 350, 361, 369, 372, 377, 378, 385, 386, 387, 391, 399, 400, 401, 403, 411, 413, 414, 417, 418, 420, 422, 425, 427, 429, 432, 440, 443, 445, 447, 452, 454, 463, 470, 484, 485, 488, 493, 502, 503, 505, 508, 517, 519, 521, 523, 525, 527, 530, 531, 534, 542, 544, 547, 551, 554, 560, 562, 563, 566, 567, 573, 576, 577, 580, 582, 590, 594, 598, 599, 600, 610, 615, 623, 628, 629, 630, 634, 644, 653, 654, 655, 656, 664, 667, 668, 673, 676, 681, 693, 696, 700, 703, 711, 712, 713, 716, 720, 721, 7

In [26]:
# End timer
end_time = time.perf_counter()
print('\n and its fitness function is:\n',int(F[0]))
print("================================================================================")
# Calculate elapsed time
elapsed_time = end_time - start_time
print("Elapsed time: ", int(elapsed_time))
print("================================================================================")
print ("Empty Clusters are:", len(EmptyClusters))
print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
print ("The number of Empty Jobs is:", len(EmptyJobs))


 and its fitness function is:
 38
Elapsed time:  59
Empty Clusters are: 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
The number of Empty Jobs is: 0
